#Aula 29 - Manipulando API's - Parte II

Antes de iniciarmos em nosso tema de hoje, vamos brincar um pouco com a API do Google para completar uma frase de sua preferência.

In [ ]:
from requests import get

frase = input('Digite uma frase: ')
url = f"http://suggestqueries.google.com/complete/search?client=firefox&q={frase}"
resp = get(url).json()
resp

Nesta aula, trabalharemos com a manipulação de dados da API gratuita da câmara dos deputados.

Endereço e documentação da API - https://dadosabertos.camara.leg.br/swagger/api.html%5D#api

Em nosso primeiro exemplo, vamos inicialmente conectar e mostrar os dados.

In [ ]:
import requests
requisicao = requests.get('https://dadosabertos.camara.leg.br/api/v2/deputados')
resposta = requisicao.json()
print(resposta)

Podemos melhorar um pouco a visualização dos dados com a estrutura for:

In [ ]:
for k in resposta['dados']:
  print(k)

Agora vamos visualizar mais claramente o nome do deputado, o seu número de identificação e o seu Estado.

In [ ]:
for r in resposta['dados']:
  print (r['nome'], r['id'], r['siglaUf'])

In [ ]:
#Listagem dos Deputados do Estado de Mato Grosso
for r in resposta['dados']:
  if r['siglaUf']=='MT':
    print (r['nome'], r['id'], r['siglaUf'])

Nós podemos manipular arquivos de uma API, em nosso exemplo, vamos aprender a fazer o download de fotos inclusas nos dados dos deputados.



In [ ]:
import requests
url = 'https://dadosabertos.camara.leg.br/api/v2/deputados'
requisicao = requests.get(url).json()
for x in requisicao['dados']:
  if x['siglaUf'] == 'MT':
    nome = x['nome'].lower()
    print (f'Gravando: {nome}')
    f = open(nome + '.jpg', 'wb')
    foto = requests.get(x['urlFoto']).content
    f.write(foto)
    f.close()

Vamos analisar o valor total gasto de um deputado federal, de acordo com a sua quota parlamentar, para isso, necessitaremos pegar o número identificador do deputado para efetuar a sua análise.

In [ ]:
id = '141401'
requisicao = requests.get(f'https://dadosabertos.camara.leg.br/api/v2/deputados/{id}/despesas?ordem=ASC&ordenarPor=ano')
resposta = requisicao.json()
soma = 0
for pg in range(1, 100):
  url = f'https://dadosabertos.camara.leg.br/api/v2/deputados/{id}/despesas?ordem=ASC&ordenarPor=ano&pagina={pg}&itens=100'
  r = requests.get(url).json()
  for gasto in r['dados']:
    soma = soma + float(gasto['valorLiquido'])#acumulador

print (f'R$ {soma:.2f}')

Agora vamos detalhar os gastos dos deputados federais, e saber onde foram efetuados os gastos, com o nome do fornecedor, o cnpj e o total gasto por fornecedor.

In [ ]:
id = '160976'
soma = 0
fornecedores = {}
cnpjCpfFornecedores = {}
for pag in range(1, 100):
  url = f'https://dadosabertos.camara.leg.br/api/v2/deputados/{id}/despesas?ordem=ASC&ordenarPor=ano&pagina={pag}&itens=100'
  r = requests.get(url).json()
  for gasto in r['dados']:
    valor = float(gasto['valorLiquido'])
    soma = soma + valor
    nome = gasto['nomeFornecedor']
    cnpjCpf = gasto['cnpjCpfFornecedor']
    if cnpjCpf not in cnpjCpfFornecedores:
      cnpjCpfFornecedores[cnpjCpf] = valor
      fornecedores[cnpjCpf] = nome
    else:
      cnpjCpfFornecedores[cnpjCpf] = cnpjCpfFornecedores[cnpjCpf] + valor
    
print (f'Total retornado API Câmara: R$ {soma:.2f}')

def chave(f): 
  return f[1]

maiores = sorted(cnpjCpfFornecedores.items(), key=chave, reverse=True)
for cnpjcpf, valor in maiores[:50]:
    print (cnpjcpf, fornecedores[cnpjcpf], f'R$ {valor:.2f}')

#Exercícios - Aula 29/06

Exercício 01 - Faça a listagem de todos os deputados, filtrando apenas o nome, o partido e o Estado.

Exercício 02 - Faça a listagem de todos os deputados a partir do Estado informado pelo usuário, filtrando apenas o nome, número de identificação, partido e sigla do Estado.

Exercício 03 - Faça a listagem de todos os deputados a partir do Partido informado pelo usuário, filtrando apenas o nome e o Estado.

Exercício 04 - Faça a pesquisa do valor total gasto por deputado, através do código de identificação informado pelo usuário.

Exercício 05 - Faça a pesquisa do valor total gasto por deputado detalhado por CNPJ, Nome do Fornecedor e Valor Gasto, através do código de identificação informado pelo usuário no input.